## What we need 

Get data from WRDS --> is this through openAP

get data from FRED? -- Where is this? 
get fama french factors? 

How do we get the long-short positions?? like how do we see the returns?

are we able to get wrds connection or should we get a file downloaded from you 

will we be able to use gridsearchCV to run multiple models at once?

In [8]:
import pandas as pd
import polars as pl
import openassetpricing as oap
import numpy as np
import wrds
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [3]:
openap = oap.OpenAP()

In [ ]:
signals = openap.dl_all_signals("polars") # need wrds
signals

OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [18]:
crsp = pd.read_csv('crsp_data.csv')
crsp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047630 entries, 0 to 4047629
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   permno  int64  
 1   date    object 
 2   ret     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 92.6+ MB


In [19]:
# crsp = pl.from_pandas(crsp)

crsp['date'] = pd.to_datetime(crsp['date'])

cleancrsp = crsp.query('ret.notnull() and date.dt.year >= 1975')
#crsp["ret"].is_not_null(), crsp['date'].dt.year() >= 1975) 
# keep only the data that has return and after 1975

len(cleancrsp['ret'].notnull()) # check that no null values in ret
len(cleancrsp['ret'].notnull()) == len(cleancrsp) # check that all ret values are not null

cleancrsp


,permno,date,ret
1,10000,1986-02-28,-25.7143
2,10000,1986-03-31,36.5385
3,10000,1986-04-30,-9.8592
4,10000,1986-05-30,-22.2656
5,10000,1986-06-30,-0.5025
...,...,...,...
4047625,93436,2024-08-30,-7.7391
4047626,93436,2024-09-30,22.1942
4047627,93436,2024-10-31,-4.5025
4047628,93436,2024-11-29,38.1469


In [ ]:
max_idx = cleancrsp['ret'].argmax()

cleancrsp.iloc[max_idx-5:max_idx+5] # this seems like an outlier for sure, not to be included in the data --> company is black stone minerals. 

,permno,date,ret
1097284,22298,2024-05-31,25.3886
1097285,22298,2024-06-28,-10.3306
1097286,22298,2024-07-31,6.9124
1097287,22298,2024-08-30,-8.6207
1097288,22298,2024-09-30,10.3774
1097289,22298,2024-10-31,3900.0000
1097290,22298,2024-11-29,-5.9829
1097291,22298,2024-12-31,-18.1364
1097293,22299,2021-12-31,0.1023
1097294,22299,2022-01-31,0.0000


In [ ]:
cleancrsp.sort_values('ret').tail(10) # check the top 10 returns
# these seem way too high

,permno,date,ret
1249683,24925,2024-05-31,1757.1428
1983399,48072,2021-01-29,1760.4650
964677,20412,2024-11-29,1791.6666
1155530,23199,2023-04-28,1832.3214
2130674,53154,1993-08-31,1900.0000
901680,19549,2024-12-31,1948.6032
674621,16400,2019-01-31,1988.3589
2276262,58748,1992-01-31,2400.0000
608386,15489,2024-05-31,2658.3828
1097289,22298,2024-10-31,3900.0000


In [ ]:
cleancrsp['permno'].nunique() # check the number of unique permnos


27071

## Open Signals from Zip

In [ ]:
from zipfile import ZipFile


def open_dataframe_from_zip(zip_filepath, csv_filename):
    """
    Opens a Pandas DataFrame from a CSV file within a zip archive.

    Args:
        zip_filepath (str): Path to the zip file.
        csv_filename (str): Name of the CSV file inside the zip archive.

    Returns:
        pandas.DataFrame: The DataFrame read from the CSV file, or None if an error occurs.
    """
    try:
        with ZipFile(zip_filepath) as zf:
            with zf.open(csv_filename) as csvf:
                df = pd.read_csv(csvf)
        return df
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_filepath}'")
        return None
    except KeyError:
        print(f"Error: CSV file '{csv_filename}' not found in the zip archive.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
zip_filepath = 'signed_predictors_dl_wide.zip'
csv_filename = 'signed_predictors_dl_wide.csv'
signals = open_dataframe_from_zip(zip_filepath, csv_filename)

if signals is not None:
    print(signals.head())
    
    

   permno  yyyymm  AM  AOP  AbnormalAccruals  Accruals  AccrualsBM  Activism1  \
0   10000  198601 NaN  NaN               NaN       NaN         NaN        NaN   
1   10000  198602 NaN  NaN               NaN       NaN         NaN        NaN   
2   10000  198603 NaN  NaN               NaN       NaN         NaN        NaN   
3   10000  198604 NaN  NaN               NaN       NaN         NaN        NaN   
4   10000  198605 NaN  NaN               NaN       NaN         NaN        NaN   

   Activism2  AdExp  ...  retConglomerate  roaq  sfe  sinAlgo  skew1  \
0        NaN    NaN  ...              NaN   NaN  NaN      NaN    NaN   
1        NaN    NaN  ...              NaN   NaN  NaN      NaN    NaN   
2        NaN    NaN  ...              NaN   NaN  NaN      NaN    NaN   
3        NaN    NaN  ...              NaN   NaN  NaN      NaN    NaN   
4        NaN    NaN  ...              NaN   NaN  NaN      NaN    NaN   

   std_turn  tang  zerotrade12M   zerotrade1M  zerotrade6M  
0       NaN   NaN  

In [12]:
signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273600 entries, 0 to 5273599
Columns: 211 entries, permno to zerotrade6M
dtypes: float64(209), int64(2)
memory usage: 8.3 GB


In [16]:
signals_exploration = signals.query('permno == 10000')

In [17]:
signals_exploration

,permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,...,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M
0,10000,198601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,198602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.785175e-08,NaN
2,10000,198603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.023392e-07,NaN
3,10000,198604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.467463e-08,NaN
4,10000,198605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.649551e-08,NaN
5,10000,198606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.360224e-08,NaN
6,10000,198607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+00,2.065574
7,10000,198608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.909091e+00,4.032000
8,10000,198609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.591646e-08,3.968504
9,10000,198610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.225312e-08,3.937500


In [21]:
cleancrsp.head(30)

,permno,date,ret
1,10000,1986-02-28,-25.7143
2,10000,1986-03-31,36.5385
3,10000,1986-04-30,-9.8592
4,10000,1986-05-30,-22.2656
5,10000,1986-06-30,-0.5025
6,10000,1986-07-31,-8.0808
7,10000,1986-08-29,-61.5385
8,10000,1986-09-30,-5.7143
9,10000,1986-10-31,-24.2424
10,10000,1986-11-28,6.0000


In [ ]:
signals = signals.rename(columns={'yyyymm': 'yyyymm_signals'})


    # Create date that is one month ahead for merging with returns
signals["yyyymm_signals"] = signals["yyyymm_signals"].astype(str) 
signals["date"] = pd.to_datetime(signals["yyyymm_signals"] + "28", format="%Y%m%d") + pd.offsets.MonthEnd(1)


## Need to Convert yyyymm column to yyyy-mm-dd datetime column, rename to date

Something interesting, the crsp dataset uses trading days. How do we get this in our signals dataset. Just go off year-months?

Can we merge on yyyy-mm and not include dd so that we can match the crsp dates? But how would it know the date in the future? For instance, would it know the end of the month a year from now, or guess based on the previous crsp dd for that year or month before?



In [ ]:
bigdata = cleancrsp.merge(signals, on=['date', 'permno'], how='left')